In [5]:
import torch
import numpy as np
import pandas as pd
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from m5_sb_models import TabNet_pipeline

def train_valid_split(data_x, data_y, train_idx, valid_idx):
    x_train = data_x.loc[train_idx].values
    y_train = data_y[train_idx].values.reshape(-1,1)
    x_valid = data_x.loc[valid_idx].values
    y_valid = data_y[valid_idx].values.reshape(-1,1)
    return x_train, y_train, x_valid, y_valid

def calculate_rmse(y, yhat):
    return mean_squared_error(y, yhat, squared=False)

def preprocess_feats(feats, scaler=StandardScaler()):
    # Replace inf/-inf with NaN and then fill NaNs with a large negative number
    feats = np.where(np.isinf(feats), np.nan, feats)
    feats = np.nan_to_num(feats, nan=-1e10)
    return scaler.fit_transform(feats)

In [6]:
train_feats = pd.read_pickle('feature_selection/train_feats.pkl')
test_feats = pd.read_pickle('feature_selection/test_feats.pkl')
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

Using cuda


In [7]:
# NEED TO DO HYPER PARAMETER TUNNING AFTER FEATS SELECTION IS COMPLETED

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

def objective(trial):
    # Define hyperparameters here as in previous examples
    rmse_scores = []
    params = {
        'n_d': trial.suggest_int('n_d', 8, 32),
        'n_a': trial.suggest_int('n_a', 8, 32),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'n_independent': trial.suggest_int('n_independent', 1, 5),
        'n_shared': trial.suggest_int('n_shared', 1, 5),
        # Include other parameters as needed
        'optimizer_fn': torch.optim.Adam,
        'scheduler_params': {"milestones": list(np.arange(50,1000,50)), 'gamma':0.2},
        'scheduler_fn': torch.optim.lr_scheduler.MultiStepLR,
    }

    test_preds, valid_preds, final_rmse, model = TabNet_pipeline(train_feats, test_feats, params)
    return np.mean(final_rmse)

Using cuda


In [8]:
import optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2023-12-31 00:04:02,432] A new study created in memory with name: no-name-156630fc-115a-4aa9-ba14-10c315605492
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/sklearn/model_selection/_split.py:737: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(


(2471, 77) (3, 77)

Early stopping occurred at epoch 213 with best_epoch = 143 and best_val_1_mse = 0.49415


/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 274 with best_epoch = 204 and best_val_1_mse = 0.54092


/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[W 2023-12-31 00:21:04,533] Trial 0 failed with parameters: {'n_d': 26, 'n_a': 9, 'n_steps': 10, 'gamma': 1.5089572467464802, 'n_independent': 5, 'n_shared': 3} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/root/miniconda3/envs/lrp/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_710091/971958586.py", line 22, in objective
    test_preds, valid_preds, final_rmse, model = TabNet_pipeline(train_feats, test_feats, params)
  File "/root/Projects/Kaggle/linking-writing/m5_sb_models.py", line 456, in TabNet_pipeline
    X_train = train_x, y_train = train_y,
  File "/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py", line 262, in fit
    self

KeyboardInterrupt: 

In [ ]:
import optuna
from sklearn.metrics import mean_squared_error
from math import sqrt

def objective(trial):
    # Hyperparameters to be tuned by Optuna
    param = {
        'n_d': trial.suggest_int('n_d', 8, 32),
        'n_a': trial.suggest_int('n_a', 8, 32),
        'n_steps': trial.suggest_int('n_steps', 3, 10),
        'gamma': trial.suggest_float('gamma', 1.0, 2.0),
        'n_independent': trial.suggest_int('n_independent', 1, 5),
        'n_shared': trial.suggest_int('n_shared', 1, 5),
        # Include other parameters as needed
        'optimizer_fn': torch.optim.Adam,
        'scheduler_params': {"milestones": [150,250,300,350,400,450], 'gamma':0.2},
        'scheduler_fn': torch.optim.lr_scheduler.MultiStepLR,
        'device_name': DEVICE
    }

    # Initialize model
    model = TabNetRegressor(**param)
    train_feats = pd.read_pickle('feature_selection/train_feats_2.pkl')
    test_feats = pd.read_pickle('feature_selection/test_feats_2.pkl')
    train_feats = preprocess_feats(train_feats)
    test_x = test_feats.drop(columns = ['id']).values
    x = train_feats.drop(['id', 'score'], axis=1)
    y = train_feats['score']

    # Splitting data (use your own train_valid_split function)
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    for i, (train_index, valid_index) in enumerate(skf.split(x, y.astype(str))):
        train_x, train_y, valid_x, valid_y = train_valid_split(x, y, train_index, valid_index)

    # Fit the model
    model.fit(
        X_train=train_x, y_train=train_y,
        eval_set=[(valid_x, valid_y)],
        max_epochs=1000,
        patience=70,
        batch_size=1024,  # or any other batch size
        virtual_batch_size=128  # or any other virtual batch size
    )

    # Evaluate the model
    valid_predictions = model.predict(valid_x)
    rmse = sqrt(mean_squared_error(valid_y, valid_predictions))

    return rmse

test_preds = []
valid_preds = pd.DataFrame()
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
final_rmse